## Hierarchical Indexing

### A Multiply Indexed Series

In [2]:
import pandas as pd
import numpy as np

#### The bad way

In [3]:
index = [('California', 2000), ('California', 2010),
        ('New York', 2000), ('New York', 2010),
        ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [4]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [5]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

#### The better way: Pandas MultiIndex

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [8]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

#### MultiIndex as extra dimension

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [11]:
pop_df = pd.DataFrame({'total': pop,
                        'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [12]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### Methods of MultiIndex Creation

In [15]:
df = pd.DataFrame(np.random.rand(4, 2), 
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.736754  0.308554
  2  0.516979  0.659719
b 1  0.868353  0.162008
  2  0.913378  0.754333

In [17]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}

pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

#### Explicit MultiIndex constructors

In [18]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'],[1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [20]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

#### MultiIndex level names

In [25]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### MultiIndex for columns

In [28]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                  names=['year', 'visit'])

columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                    names=['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  37.5  26.0  34.8  35.0  36.7
     2      29.0  37.2  17.0  37.0  31.0  36.2
2014 1      35.0  38.3  37.0  37.3  32.0  37.6
     2      37.0  36.4  39.0  36.3  28.0  35.7

In [31]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      26.0  34.8
     2      17.0  37.0
2014 1      37.0  37.3
     2      39.0  36.3

### Indexing and Slicing a MultiIndex

#### Multiply indexed Series

In [32]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [33]:
pop['California', 2000]

33871648

In [34]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [35]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [36]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [39]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [40]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Multiply indexed DataFrames

In [41]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  37.5  26.0  34.8  35.0  36.7
     2      29.0  37.2  17.0  37.0  31.0  36.2
2014 1      35.0  38.3  37.0  37.3  32.0  37.6
     2      37.0  36.4  39.0  36.3  28.0  35.7

In [42]:
health_data['Guido', 'HR']

year  visit
2013  1        26.0
      2        17.0
2014  1        37.0
      2        39.0
Name: (Guido, HR), dtype: float64

In [43]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      31.0  37.5
     2      29.0  37.2

In [44]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        31.0
      2        29.0
2014  1        35.0
      2        37.0
Name: (Bob, HR), dtype: float64

In [45]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-45-fb34fa30ac09>, line 1)

In [48]:
# Using an IndexSlice object to access the desired data
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,31.0,26.0,35.0
2014,1,35.0,37.0,32.0


### Rearranging Multi-Indices

#### Sorted and unsorted indices

In [49]:
# Many of the MultiIndex slicing operations will fail if the index is not sorted!
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.376038
      2      0.529776
c     1      0.038992
      2      0.320149
b     1      0.453531
      2      0.802663
dtype: float64

In [51]:
# Exception raised because MultiIndex is not sorted!
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [52]:
data = data.sort_index()
data

char  int
a     1      0.376038
      2      0.529776
b     1      0.453531
      2      0.802663
c     1      0.038992
      2      0.320149
dtype: float64

In [54]:
# Now that the data is properly sorted, slicing the data works fine!
data['a':'b']

char  int
a     1      0.376038
      2      0.529776
b     1      0.453531
      2      0.802663
dtype: float64

#### Stacking and unstacking indices

In [55]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [56]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [57]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### Index setting and resetting

In [60]:
# reset_index generates columns for the previous values set as labels
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [59]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [62]:
# Setting both previously set labels as indexes
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### Data Aggregations on Multi-Indices

In [63]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  37.5  26.0  34.8  35.0  36.7
     2      29.0  37.2  17.0  37.0  31.0  36.2
2014 1      35.0  38.3  37.0  37.3  32.0  37.6
     2      37.0  36.4  39.0  36.3  28.0  35.7

In [65]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido         Sue       
type       HR   Temp    HR  Temp    HR   Temp
year                                         
2013     30.0  37.35  21.5  35.9  33.0  36.45
2014     36.0  37.35  38.0  36.8  30.0  36.65

In [ ]:
# Specifying aggregation on a column
data_mean.mean(axis=1)